### https://towardsdatascience.com/linear-discriminant-analysis-in-python-76b8b17817c2

# LOADING LIBRARIES

In [6]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_wine
np.set_printoptions(precision=4)
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

# Dataset 

In [29]:
with open('../Simulation/multiple_sqrt_sim_data_pos.json') as f:
    full_data_dict = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
f1_scores = {}
avg_nb_corner = 0
nb_2corners = 0
nb_sim = 0

for pose in poses_drf:
    for gap in finger_gaps:
        for dim in sqrt_sides:
            dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)
            data_dict = full_data_dict[dict_key]

            data_list = [data_dict['LF_motion1'],data_dict['RF_motion1']]
            data_array = np.array(data_list).T
            temp_data_array = np.append(data_array[np.newaxis,0,:,0],data_array[np.newaxis,0,:,1],axis=0)
            data_array = np.append(temp_data_array,data_array[np.newaxis,1,:,1],axis=0).T
            angle_array = data_array[:,:2].astype(float)

            # CREATE A DATAFRAME BY WINDOWING THE RAW AND INCREMENTING THE POSITION OF THE WINDOW
            rows,columns = data_array.shape
            window_size = 5 # needs to be odd
            half_window_size = math.floor(window_size/2)
            data_window = np.zeros((rows-half_window_size*2,window_size*2+1)) #+1 for the class
            corners = []
            sides = []

            first_corner_hit = False
            second_corner_hit = False
            for i in range(half_window_size,rows-half_window_size):
                temp_window = np.append(data_array[i-half_window_size:half_window_size+i+1,:].T[0],
                                        data_array[i-half_window_size:half_window_size+i+1,:].T[1])
                # class 1 --> side1
                if 'corner' not in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[2] and not first_corner_hit and not second_corner_hit:
                    sides += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([1]))
                # class 2 --> corner1
                elif data_array[i-half_window_size:half_window_size+i+1,:].T[2][-2] == 'corner' and not first_corner_hit and not second_corner_hit:
                    first_corner_hit = True
                    corners += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([2]))
                # class 4 --> corner2
                elif data_array[i-half_window_size:half_window_size+i+1,:].T[2][-2] == 'corner' and first_corner_hit and not second_corner_hit:
                    second_corner_hit = True
                    corners += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([4]))
                # class 2 --> corner1
                elif 'corner' in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[2] and first_corner_hit and not second_corner_hit:
                    corners += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([2]))
                # class 3 --> side2
                elif 'corner' not in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[2] and first_corner_hit and not second_corner_hit:
                    sides += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([3]))
                # class 4 --> corner2
                elif 'corner' in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[2] and first_corner_hit and second_corner_hit:
                    corners += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([4]))
                # class 5 --> side3
                else :
                    sides += [angle_array[i].tolist()]
                    data_window[i-half_window_size] = np.append(temp_window,np.array([5]))

            #print(data_window)
            names = []
            for j in range(window_size):
                names += ["angleLF"+str(j+1)]
            for j in range(window_size):
                names += ["angleRF"+str(j+1)]
            names += ["class"]

            # DATA FRAME PANDAS
            df = pd.DataFrame(data=data_window,columns = names)
            df["class"].replace({1.0: "side1", 2.0: "corner1" ,3.0: "side2", 4.0: "corner2",5.0: "side3"}, inplace=True)
            df.dropna(how='all', inplace=True)
            windows = df[names[:len(names)-1]]
            labels = df[names[-1]]

            data_window_array = np.array(data_window)
            corners_pos = np.array(corners)
            sides_pos = np.array(sides)

            df.head()
            #windows.head()
            #labels.hed()

            ################## LDA Sklearn ##################

            # Preparation of the data for LDA
            # Standardizing data
            X = df.iloc[:, 0:-1].values
            le = LabelEncoder()
            y = le.fit_transform(df['class'])
            #print("After formating the data we have: \n X = \n",X[:5],'\n y = \n', y)

            # Import LDA from sklearn
            from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
            lda = LinearDiscriminantAnalysis()
            lda_X = lda.fit_transform(X, y)
            #print("The explained variance ratio of lda is : ", lda.explained_variance_ratio_)

            ###################  ELBOW METHOD ###################

            distortions = []
            inertias = []
            mapping1 = {}
            mapping2 = {}
            K = range(1, 10)

            for k in K:
                # Building and fitting the model
                kmeanModel = KMeans(n_clusters=k, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
                y_kmeans = kmeanModel.fit_predict(lda_X)

                distortions.append(sum(np.min(cdist(lda_X, kmeanModel.cluster_centers_,
                                                    'euclidean'), axis=1)) / lda_X.shape[0])
                inertias.append(kmeanModel.inertia_)
                mapping1[k] = sum(np.min(cdist(lda_X, kmeanModel.cluster_centers_,
                                               'euclidean'), axis=1)) / lda_X.shape[0]
                mapping2[k] = kmeanModel.inertia_

            ################### INERTIAS ###################

            #for key, val in mapping2.items():
            #    print(f'{key} : {val}')

            print("---------")
            print("Inertias decrease")
            print("% decrease = 100 x (initial - current)/current")
            predicted_corner_number = 0
            threshold = 27
            
            for i in range(1,len(inertias)):
                if (inertias[i-1]-inertias[i])/inertias[i-1]*100 < threshold:
                    predicted_corner_number = i-3
                    break
                print(i+1," : ",(inertias[i-1]-inertias[i])/inertias[i-1]*100)
            print("------")  
            print("The predicted number of corners is : ",predicted_corner_number)

            """SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8) 

            plt.plot(K, inertias, 'bx-')
            plt.xlabel('Values of K')
            plt.ylabel('Inertia')
            plt.title('The Elbow Method using Inertia', fontsize = 20)"""
            if predicted_corner_number == 2:
                nb_2corners +=1
            avg_nb_corner += predicted_corner_number
            nb_sim +=1


---------
Inertias decrease
% decrease = 100 x (initial - current)/current
2  :  90.39361655317796
3  :  43.8671137567743
4  :  63.954506645548
5  :  46.8228905248889
------
The predicted number of corners is :  2
---------
Inertias decrease
% decrease = 100 x (initial - current)/current
2  :  92.54047234881112
3  :  70.67046208486335
4  :  38.837490485060414
5  :  35.64620297188782
6  :  35.38802350110551
7  :  31.789639146226744
8  :  35.021063212873365
9  :  29.031485131405567
------
The predicted number of corners is :  0
---------
Inertias decrease
% decrease = 100 x (initial - current)/current
2  :  85.77346135532807
3  :  58.13259822050287
4  :  71.37065838646767
5  :  30.99467176729424
6  :  38.56542354124405
7  :  49.707720528474724
8  :  80.25802058617705
9  :  42.029593046257304
------
The predicted number of corners is :  0
---------
Inertias decrease
% decrease = 100 x (initial - current)/current
2  :  91.35064957620168
3  :  87.10131880171157
4  :  43.627897435911294
----

In [30]:
print("the average number of corner counted during the manipulation is : ",avg_nb_corner/nb_sim)
print(nb_2corners)
print(nb_sim)
print(nb_2corners/nb_sim*100)

the average number of corner counted during the manipulation is :  1.5925925925925926
20
27
74.07407407407408


## Decision Trees

https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

## Support Vector Machine

## Naive Bayes

# In the case you want to do it manually

In [45]:
labels_feature_means = pd.DataFrame(columns=np.array(["side","corner"]))

for c, rows in df.groupby('class'):
    labels_feature_means[c] = rows.mean()
labels_feature_means 

,side,corner
angleLF1,87.869048,75.000000
angleLF2,88.869048,76.000000
angleLF3,89.869048,77.000000
angleLF4,90.869048,78.000000
angleLF5,91.869048,79.000000
angleRF1,95.710595,90.482112
angleRF2,96.715493,91.838239
angleRF3,97.724740,93.085030
angleRF4,98.738635,94.220319
angleRF5,99.757535,95.241369
